<a href="https://colab.research.google.com/github/PariaNajafi/Deep-Learning-BootCamp/blob/main/OnAcademy_DeepLearning_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Sentiment Classification(analysis) Using RNNs

In [6]:
#https://ai.stanford.edu/~amaas/sentiment/

In [7]:
!pip install datasets -q

In [8]:
#torchtext,datasets

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab') # for tokenize
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from torch.utils.data import DataLoader,TensorDataset
from torch.nn.utils.rnn import pad_sequence

print("PyTorch Version:", torch.__version__)

PyTorch Version: 2.6.0+cu124


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [12]:
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")

Using Device: cpu


Classification, labels:0,1 , comment says the film is good or bad

##Load and Preprocess IMDB Dataset

In [11]:
dataset = load_dataset("imdb")
train_data = dataset["train"]
test_data = dataset["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

We need to convert word and sentence data into numerical equivalents.
hello : 0,
the : 5,
, : 4

In [13]:
def build_vocab(texts, max_vocab_size=10000):
    all_tokens = []
    for text in texts:
        tokens = word_tokenize(text.lower())
        all_tokens.extend(tokens)
    token_counts = Counter(all_tokens)
    # Sort tokens by frequency in descending order by making a dictionary
    vocab = {'<pad>' : 0, '<unk>' : 1}
    vocab.update({token : ind + 2 for ind, (token, _) in enumerate(token_counts.most_common(max_vocab_size - 2))})
    return vocab


In [17]:
vocab = build_vocab(train_data["text"])
#how manay words exicts in vocabulary
print(f"Vocabulary Size: {len(vocab)}")
#now we have made numeral ecuivalants of words, now we have a dictionary, any word out of this dictionary equivalants with unknown

Vocabulary Size: 10000


In [18]:
#comment classifies the film in good or bad classes in first 100 words.
def text_to_indices(text, vocab,max_len=100):
    tokens = word_tokenize(text.lower())[:max_len]
    indices = [vocab.get(token, vocab['<unk>']) for token in tokens]

    return indices


In [19]:
def process_data(data,vocab, max_len=100):
  texts = [text_to_indices(text, vocab, max_len) for text in data['text']]
  labels = [label for label in data['label']]
  3# Pad sequences
  padded_texts = [torch.tensor(text, dtype=torch.long) for text in texts]
  padded_texts = pad_sequence(padded_texts, batch_first=True, padding_value=0)
  labels = torch.tensor(labels, dtype=torch.float32)

  return padded_texts, labels

In [20]:
train_texts, train_labels = process_data(train_data, vocab)
test_texts, test_labels = process_data(test_data, vocab)
train_dataset = TensorDataset(train_texts, train_labels)
test_dataset = TensorDataset(test_texts, test_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

####test

In [21]:
train_data[0]['text']
#train data is a dictionary with 2 elements

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [22]:

all_tokens = []
for text in train_data["text"]:
        tokens = word_tokenize(text.lower())
        all_tokens.extend(tokens)
token_counts.most_common(5)


NameError: name 'token_counts' is not defined

In [ ]:
a= [torch.tensor([5, 6, 7]), torch.tensor([5, 6, 7, 10, 12, 15])]
pad_sequence(a, padding_value=0)

#Build Model

In [ ]:
class SentimentLSTM(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
    super(SentimentLSTM, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim,padding_ind=0)
    self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True) #output = hidden_dim
    self.fc = nn.Linear(hidden_dim, output_dim)
    self.sigmoid = nn.Sigmoid()
  def forward(self, text):
    embedded = self.embedding(text) #[batch, seq_len, embed_dim]
    _, (hidden, _) = self.lstm(embedded) #hidden: [1, batch, hidden_dim] ,_ : all LSTMs outputs,  the last blocks output: (hidden, _)
    hidden = hidden.squeeze(0) #[batch, hidden_dim]
    output = self.fc(hidden) # [batch, output_dim]
    return self.sigmoid(output)
    #2 class classification passes to sigmoid activation, >0.5 : 1 , <0.5 : 0
    # we dont pass the lenght to RNNs, it handles it itself

  # Model parameters
  VOCAB_SIZE = len(vocab)
  EMBED_DIM = 50
  HIDDEN_DIM = 128
  OUTPUT_DIM = 1

  model = SentimentLSTM(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, OUTPUT_DIM)
  model.to(device)
  print(model)

####test

In [ ]:
aa = nn.embedding = nn.Embedding(1000,150 ,padding_ind=0)
(vocab_size,embed_dim )
# max value of numbers = 1000
# mbed_dim is the length of the array
# padding_ind=0  dont do back propagation or deviation for this part of data, dont update 0 values they are padding and must have no effect
# in one hot encoding value was specific but in embedding it is not specific and it specifies in the process, its not a discrete array full of 0 and 1, they are between 2 number
for i in train_texts:
  a = aa(i)
  print(a)
  print(a.shape)
  break

#[100,150] sequence size(tedad vajegan jomle), each word equevalents an array with 150 elements.150 depends on systems Computational Limitation,
#optimized size depends on size of the dictionary whitch here is 1000

#Train Model

In [ ]:
critrion = nn.BCELoss() #binary cross entropy loss function uses in  binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs=3)
  model.train()
  losses, accuracies = [], []
  for epoch in range(epochs):
    epoch_loss, epoch_correct, epoch_total =0, 0, 0
    for text, labels in train_loader:
      text, labels = text.to(device), labels.to(device)
      optimizer.zero_grad()
      predictions = model(text).squeeze(1)
      loss = criterion(predictions, labels)
      loss.backward()
      optimizer.step()

      epoch_loss += loss.item()
      predicted_labels = (predictions > 0.5).float()
      epoch_correct += (predicted_labels == labels).sum().item()
      epoch_total += labels.size(0)

    avg_loss = epoch_loss / len(train_loader)
    avg_acc = epoch_correct / epoch_total
    losses.append(avg_loss)
    accuracies.append(avg_acc)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}")
  return losses, accuracies

#Train the model
train_losses, train_accuracies = train_model(model, train_loader, criterion, optimizer, epochs = 50)


In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()

plt.tight_layout
plt.savefig('training_metrics.png')
plt.show()

# Evaluation on Test Set

In [ ]:
def evaluate_model(model, test_loader):
  model.eval()
  correct, total = 0, 0
  with torch.no_grad():
    for text, labels in test_loader:
      text, labels = text.to(device), labels.to(device)
      predictions = model(text).squeeze(1)
      predicted_labels = (predictions > 0.5).float()
      correct += (predicted_labels == labels).sum().item()
      total += labels.size(0)
  accuracy = correct / total

  print(f"Test Accuracy: {accuracy:.4f}")
  return accuracy

# Evaluate
test_accuracy = evaluate_model(model, test_loader)
